# 06: Postprocess data

In this script, we postprocess the raw LCIA impact data:
- split off production volumes
- Filter out failed computations
- convert to equal product units (MJ)
- add corresponding IAM regions
- average using production volumes
- combine production and combustion impacts in the case of liquid fuels and hydrogen + gases
- calculate indirect impacts

In [1]:
%run common_definitions.py

In [2]:
import pandas as pd
import numpy as np
import xarray as xr

from premise.geomap import Geomap

In [3]:
output_fp = "../output/" + BW_PROJECTNAME

### Load impact file

In [4]:
impacts = pd.read_csv(output_fp+"/impacts_energy_provision.csv")
direct_impacts = pd.read_csv(output_fp + "/direct_impacts_energy_provision.csv")

### Split off production volumes, fill missing values

In [5]:
prod_volumes = impacts.copy().set_index(["short name", "sector", "scenario", "year", "location"])["production volume"]

To allow weighting by production volumes, we fill missing values:

- In the case of the overall average, we fill with ones in cases where values are missing or zero for all locations.
- In the case of the IAM region averages, we fill with ones in cases where values are missing or zero for all locations within a IAM region

In [6]:
geo = Geomap(model="remind")

prod_volumes = prod_volumes.reset_index()
prod_volumes["IAM region"] = prod_volumes["location"].apply(geo.ecoinvent_to_iam_location)
prod_volumes.set_index(["short name", "sector", "scenario", "year", "location"], inplace=True)

In [7]:
def count_valid(x):
    return np.count_nonzero(np.nan_to_num(x))

In [8]:
nan_counts = prod_volumes.reset_index().groupby(["short name", "sector", "scenario", "year"]).agg({
    "production volume": count_valid
})
fill_idx = []
compare_idx = nan_counts[nan_counts["production volume"] == 0].index
for i, idx in enumerate(prod_volumes.reset_index().set_index(["short name", "sector", "scenario", "year"]).index):
    if idx in compare_idx:
        fill_idx.append(i)

s1 = prod_volumes["production volume"].copy()
s1.iloc[fill_idx] = 1.0

In [9]:
nan_counts = prod_volumes.reset_index().groupby(["short name", "sector", "scenario", "year", "IAM region"]).agg({
    "production volume": count_valid
})
fill_idx = []
compare_idx = nan_counts[nan_counts["production volume"] == 0].index
for i, idx in enumerate(prod_volumes.reset_index().set_index(["short name", "sector", "scenario", "year", "IAM region"]).index):
    if idx in compare_idx:
        fill_idx.append(i)

s2 = prod_volumes["production volume"].copy()
s2.iloc[fill_idx] = 1.0

In [10]:
prod_volumes["production volume (for overall average)"] = s1
prod_volumes["production volume (for IAM region averages)"] = s2

Fill the rest with zeros.

In [11]:
prod_volumes.fillna(0, inplace=True)
prod_volumes.to_csv(output_fp + "/production_volumes_energy_provision.csv")

### Filter out failed computations (only for full impacts)

In [12]:
impacts.drop(["amount", "unit", "ecoinvent name", "production volume"], axis=1, inplace=True)
direct_impacts.drop(["amount", "unit", "ecoinvent name"], axis=1, inplace=True)

impacts.set_index(["short name", "product", "sector", "scenario", "year", "location"], inplace=True)

failed = impacts.loc[~impacts.any(axis=1), :]
impacts.drop(failed.index, inplace=True)
impacts.reset_index(inplace=True)

### Convert to impacts per MJ

In [13]:
NCVs = pd.read_csv("../data/NCVs_v2.csv")
NCVs["conversion factor"] = np.where(~np.isnan(NCVs["NCV in MJ/product"]), 1/NCVs["NCV in MJ/product"],
                                     np.where(~np.isnan(NCVs["GCV in MJ/product"]), 1/NCVs["GCV in MJ/product"], 1))
conv_factors = dict(zip(list(NCVs["product name"]), list(NCVs["conversion factor"])))
conversion_factors = impacts["product"].apply(lambda x: conv_factors[x]).to_numpy()
conversion_factors_direct = direct_impacts["product"].apply(lambda x: conv_factors[x]).to_numpy()

impacts.set_index(["short name", "product", "sector", "scenario", "year", "location"], inplace=True)
direct_impacts.set_index(["short name", "product", "sector", "scenario", "year", "location"], inplace=True)
conversion_factors = pd.Series(conversion_factors, impacts.index)
conversion_factors_direct = pd.Series(conversion_factors_direct, direct_impacts.index)

rescaled_impacts = impacts.mul(conversion_factors, axis=0)
rescaled_direct_impacts = direct_impacts.mul(conversion_factors_direct, axis=0)

rescaled_impacts = rescaled_impacts.droplevel("product", axis=0)
rescaled_direct_impacts = rescaled_direct_impacts.droplevel("product", axis=0)

### Filter out outliers

In [14]:
idx2remove = [idx for idx in rescaled_impacts.index if
              idx[0] == "hydro, reservoir, alpine region" and
              idx[-1] == "PE"]

rescaled_impacts.drop(idx2remove, axis=0, inplace=True)

idx2remove = [idx for idx in rescaled_direct_impacts.index if
              idx[0] == "hydro, reservoir, alpine region" and
              idx[-1] == "PE"]

rescaled_direct_impacts.drop(idx2remove, axis=0, inplace=True)

### Combine with combustion impacts, save

In [15]:
mapping = pd.read_csv("../../mappings/technology_selection_energy_provision_v4.csv")
short2long = dict(zip(list(mapping["short name"]), list(mapping["ecoinvent name"])))

In [16]:
combustion_impacts1 = pd.read_csv(output_fp+"/impacts_liquids_combustion.csv").set_index(["ecoinvent name"])

data = []
sel = rescaled_impacts.copy().reset_index()
sel = sel[sel["sector"] == "liquids"]
for tech in sel["short name"]:
    data.append(combustion_impacts1.loc[short2long[tech]].to_numpy())

sel.set_index(["short name", "scenario", "year", "location"], inplace=True)

rescaled_combustion_impacts1 = pd.DataFrame(
    np.array(data),
    index=sel.index,
    columns=combustion_impacts1.columns
)

rescaled_combustion_impacts1.reset_index(inplace=True)
rescaled_combustion_impacts1["sector"] = "liquids"

In [17]:
combustion_impacts2 = pd.read_csv(output_fp+"/impacts_hydrogen_and_gases_combustion.csv").set_index(["ecoinvent name"])

data = []
sel = rescaled_impacts.copy().reset_index()
sel = sel[sel["sector"] == "hydrogen and gases"]
for tech in sel["short name"]:
    data.append(combustion_impacts2.loc[short2long[tech]].to_numpy())

sel.set_index(["short name", "scenario", "year", "location"], inplace=True)

rescaled_combustion_impacts2 = pd.DataFrame(
    np.array(data),
    index=sel.index,
    columns=combustion_impacts2.columns
)

rescaled_combustion_impacts2.reset_index(inplace=True)
rescaled_combustion_impacts2["sector"] = "hydrogen and gases"

In [18]:
rescaled_combustion_impacts1.set_index(
    ["short name", "sector", "scenario", "year", "location"],
    inplace=True
)
rescaled_combustion_impacts2.set_index(
    ["short name", "sector", "scenario", "year", "location"],
    inplace=True
)
rescaled_combustion_impacts = pd.concat([rescaled_combustion_impacts1, rescaled_combustion_impacts2])
# rescaled_combustion_impacts.set_index(
#     ["short name", "sector", "scenario", "year", "location"],
#     inplace=True
# )
combined_impacts = rescaled_impacts.add(rescaled_combustion_impacts, fill_value=0)
combined_impacts.to_csv(output_fp+"/impacts_combined.csv")

### Collect all direct impacts

In [19]:
all_rescaled_direct_impacts = pd.concat([rescaled_direct_impacts, rescaled_combustion_impacts])
all_rescaled_direct_impacts.to_csv(output_fp+"/direct_impacts.csv")

### Calculate indirect impacts

In [20]:
rescaled_impacts

CML v4.8 2016, acidification, acidification (incl. fate, average Europe total, A&B)  \
short name                     sector                 scenario       year location                                                                                        
BIGCC                          electricity production SSP2-NPi       2020 CAZ                                                0.000082                                     
                                                                          CHA                                                0.000088                                     
                                                                          MEA                                                0.000086                                     
                                                                          IND                                                0.000090                                     
                                                                          LAM                                                0.000083                                     
...                                                                                                                               ...                                     
diffusion adsorption heat pump residential heating    SSP2-PkBudg500 2050 LAM                                                0.000028                                     
                                                                          CAZ                                                0.000024                                     
                                                                          CHA                                                0.000029                                     
                                                                          IND                                                0.000029                                     
                                                                          REF                                                0.000029                                     

                                                                                    CML v4.8 2016, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels  \
short name                     sector                 scenario       year location                                                                                                    
BIGCC                          electricity production SSP2-NPi       2020 CAZ                                                0.123604                                                 
                                                                          CHA                                                0.153080                                                 
                                                                          MEA                                                0.140215                                                 
                                                                          IND                                                0.149496                                                 
                                                                          LAM                                                0.126498                                                 
...                                                                                                                               ...                                                 
diffusion adsorption heat pump residential heating    SSP2-PkBudg500 2050 LAM                                                0.793124                                                 
                                                                          CAZ                                                0.737421                                                 
                              

In [21]:
rescaled_indirect_impacts = combined_impacts.sub(all_rescaled_direct_impacts)
rescaled_indirect_impacts.to_csv(output_fp+"/indirect_impacts.csv")

### Add IAM locations, calculate several regional averages

In [22]:
def grouped_weighted_avg(values, weights, weights_col, by):
    A = (values.mul(weights[weights_col], axis=0)).reset_index().groupby(by).sum(numeric_only=True)
    b = weights.reset_index().groupby(by).sum(numeric_only=True)
    return A.div(b[weights_col], axis=0)

In [23]:
geo = Geomap(model="remind")

combined_impacts.reset_index(inplace=True)
all_rescaled_direct_impacts.reset_index(inplace=True)
rescaled_indirect_impacts.reset_index(inplace=True)
prod_volumes = prod_volumes.reset_index()

combined_impacts["IAM region"] = combined_impacts["location"].apply(geo.ecoinvent_to_iam_location)
all_rescaled_direct_impacts["IAM region"] = all_rescaled_direct_impacts["location"].apply(geo.ecoinvent_to_iam_location)
rescaled_indirect_impacts["IAM region"] = rescaled_indirect_impacts["location"].apply(geo.ecoinvent_to_iam_location)
prod_volumes["IAM region"] = prod_volumes["location"].apply(geo.ecoinvent_to_iam_location)

combined_impacts.set_index(["short name", "sector", "scenario", "year", "IAM region", "location"], inplace=True)
all_rescaled_direct_impacts.set_index(["short name", "sector", "scenario", "year", "IAM region", "location"], inplace=True)
rescaled_indirect_impacts.set_index(["short name", "sector", "scenario", "year", "IAM region", "location"], inplace=True)
prod_volumes.set_index(["short name", "sector", "scenario", "year", "IAM region", "location"], inplace=True)

In [24]:
overall_average = grouped_weighted_avg(combined_impacts, prod_volumes, "production volume (for overall average)",
                                       by=["short name", "sector", "scenario", "year"])
overall_average["regional average"] = "overall"
overall_average_direct = grouped_weighted_avg(all_rescaled_direct_impacts, prod_volumes, "production volume (for overall average)",
                                       by=["short name", "sector", "scenario", "year"])
overall_average_direct["regional average"] = "overall"
overall_average_indirect = grouped_weighted_avg(rescaled_indirect_impacts, prod_volumes, "production volume (for overall average)",
                                       by=["short name", "sector", "scenario", "year"])
overall_average_indirect["regional average"] = "overall"

For the average by IAM regions, we need to adapt the production volumes in the case of only NaN 

In [25]:
# full impacts
iam_average = grouped_weighted_avg(combined_impacts, prod_volumes, "production volume (for IAM region averages)",
                                   by=["short name", "sector", "scenario", "year", "IAM region"])
iam_average.reset_index(inplace=True)

world_average = iam_average.groupby(["short name", "sector", "scenario", "year"]).mean(numeric_only=True)
world_average["regional average"] = "World"

iam_average = iam_average[iam_average["IAM region"].isin(REMIND_REGIONS)]
iam_average.rename(columns={"IAM region": "regional average"}, inplace=True)
iam_average.set_index(["short name", "sector", "scenario", "year", "regional average"], inplace=True)

# direct impacts
iam_average_direct = grouped_weighted_avg(all_rescaled_direct_impacts, prod_volumes, "production volume (for IAM region averages)",
                                   by=["short name", "sector", "scenario", "year", "IAM region"])
iam_average_direct.reset_index(inplace=True)

world_average_direct = iam_average_direct.groupby(["short name", "sector", "scenario", "year"]).mean(numeric_only=True)
world_average_direct["regional average"] = "World"

iam_average_direct = iam_average_direct[iam_average_direct["IAM region"].isin(REMIND_REGIONS)]
iam_average_direct.rename(columns={"IAM region": "regional average"}, inplace=True)
iam_average_direct.set_index(["short name", "sector", "scenario", "year", "regional average"], inplace=True)

# indirect impacts
iam_average_indirect = grouped_weighted_avg(rescaled_indirect_impacts, prod_volumes, "production volume (for IAM region averages)",
                                   by=["short name", "sector", "scenario", "year", "IAM region"])
iam_average_indirect.reset_index(inplace=True)

world_average_indirect = iam_average_indirect.groupby(["short name", "sector", "scenario", "year"]).mean(numeric_only=True)
world_average_indirect["regional average"] = "World"

iam_average_indirect = iam_average_indirect[iam_average_indirect["IAM region"].isin(REMIND_REGIONS)]
iam_average_indirect.rename(columns={"IAM region": "regional average"}, inplace=True)
iam_average_indirect.set_index(["short name", "sector", "scenario", "year", "regional average"], inplace=True)

In [26]:
# full impacts
pd.concat(
    [
        overall_average.reset_index(),
        iam_average.reset_index(),
        world_average.reset_index(),
    ],
    axis=0
).set_index(
    ["short name", "sector", "scenario", "year", "regional average"]
).sort_index().to_csv(output_fp + "/impacts_regional_averages.csv")

# direct impacts
pd.concat(
    [
        overall_average_direct.reset_index(),
        iam_average_direct.reset_index(),
        world_average_direct.reset_index(),
    ],
    axis=0
).set_index(
    ["short name", "sector", "scenario", "year", "regional average"]
).sort_index().to_csv(output_fp + "/direct_impacts_regional_averages.csv")

# indirect impacts
pd.concat(
    [
        overall_average_indirect.reset_index(),
        iam_average_indirect.reset_index(),
        world_average_indirect.reset_index(),
    ],
    axis=0
).set_index(
    ["short name", "sector", "scenario", "year", "regional average"]
).sort_index().to_csv(output_fp + "/indirect_impacts_regional_averages.csv")